# Importation des Package

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import bs4
import re
import copy
import collections as co
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np


# Webscrapping des notes ImDB des 250 mieux notés films et séries


On crée une fonction qui met dans un data frame les notes des 250 mieux notés sur ImDB (films ou séries)

In [5]:
def recup_notes(url):

    """On récupère la page internet------------------"""
    from urllib import request
    request_text = request.urlopen(url).read()
    #print(request_text[:1000]) 
    page = bs4.BeautifulSoup(request_text, "lxml")
    #print(type(page))


    """On récupère les données------------------"""
    tableau=page.find("table",{'class' : 'chart full-width'})
    #print(tableau)
    table_body=tableau.find('tbody')
    rows = table_body.find_all('tr')
    #print (rows)


    """On récupére les élements correspondant au titre et à la note--------------"""
    new_rows=[] #on crée une liste vide dans laquelle on va mettre le contenu des lignes
    for row in rows:
        cols = row.find_all('td')
        #print(type(cols))
        new_cols = []
        for ele in cols:
            new_cols.append([ele.text.strip()])
        #print(new_cols)
        new_cols=new_cols[1:3]
        #new_cols_title=new_cols[0]
        new_cols[0]= new_cols[0][0]
        new_cols[1]= float(new_cols[1][0])
        new_rows.append(new_cols)
  #print (new_rows)

    """On nettoie les élements pour n'avoir que des listes de titres et notes"""
    """L'élement correspondant au titre est entouré d'informations parasites, on s'en
    débarasse avec 2 regex successives"""

    regex_debut = "[\d]?[\d]?[\d]\.[\n][\s]{6}"
    regex_fin="[\n]\([1-2][\d]{3}\)"

    clean_rows=copy.copy(new_rows)

    for i in range(len(new_rows)):
        a_suppr=re.findall(regex_debut,new_rows[i][0])
        #print (a_suppr[0])
        clean_rows[i][0]=new_rows[i][0].replace(a_suppr[0],"")
    #print (clean_rows)

    really_clean_rows=copy.copy(clean_rows)

    for i in range(len(clean_rows)):
        a_suppr_fin=re.findall(regex_fin,clean_rows[i][0])
        #print (a_suppr_fin[0])
        really_clean_rows[i][0]=clean_rows[i][0].replace(a_suppr_fin[0],"")
    #print (really_clean_rows)

    """On crée un dictionnaire dans lequel on met le contenu des lignes nettoyées
    L'index est le titre, la valeur la note."""
    dico_top = dict()
    for row in really_clean_rows:
        cols=row
        dico_top[cols[0]] = cols[1:]
    dico_top

    """On crée ENFIN le dataframe"""
    df_top = pd.DataFrame.from_dict(dico_top,orient='index')
  

    """"On rajoute une index différente du titre"""
    df_top.reset_index(level=0, inplace=True)

    """On change le nom des colonnes"""
    df_top.rename(columns={'index': 'Title', 0: 'IMDb rating'}, inplace=True)

    #print(df_top)
    return (df_top)


Test sur les 2 urls

In [7]:
url_top_shows = "https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250"
url_top_movies= "https://www.imdb.com/chart/top/?ref_=nv_mv_250"

In [358]:
df_top_shows=recup_notes(url_top_shows)
df_top_shows
#problème : on a que 245 valeurs ...

,Title,IMDb rating
0,Planet Earth II,9.5
1,Planète Terre,9.4
2,Frères d'armes,9.4
3,Breaking Bad,9.4
4,Chernobyl,9.4
...,...,...
239,Community,8.4
240,Wentworth,8.4
241,Vikings,8.4
242,"Unsere Mütter, unsere Väter",8.4


In [9]:
df_top_movies=recup_notes(url_top_movies)
df_top_movies


,Title,IMDb rating
0,Les évadés,9.2
1,Le parrain,9.1
2,"Le parrain, 2ème partie",9.0
3,The Dark Knight: Le chevalier noir,9.0
4,12 hommes en colère,8.9
...,...,...
244,Princess Bride,8.0
245,La bataille d'Alger,8.0
246,Mandarines,8.0
247,Le cirque,8.0
